In [ ]:
from flask import Flask, jsonify, redirect, request
from flask_cors import CORS
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from konlpy.tag import Okt
import json

#커스텀 모델 호출을 위한 함수 선언
from keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림 - 0.5 이하 = 0 / 0.5 초과 = 1
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정
    # True Positive = 실제 값과 예측 값이 모두 1(Positive)인 경우
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)
    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon() = 'divide by zero error' 예방차원에서 작은 수를 더함
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    # return a single tensor value
    return recall
def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림 - 0.5 이하 = 0 / 0.5 초과 = 1
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정
    # True Positive = 실제 값과 예측 값이 모두 1(Positive)인 경우
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)
    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon() = 'divide by zero error' 예방차원에서 작은 수를 더함
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())
    # return a single tensor value
    return precision
def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon() = divide by zero error' 예방차원에서 작은 수를 더함
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    # return a single tensor value
    return _f1score

# Server
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

# Ai Function 
@app.route("/use_model", methods=['POST'])
def use_model():
	# Chrome Extension으로부터 전송받은 데이터
	examples_temp = request.get_json()
	examples = examples_temp["text"]
	check = examples_temp["check"]
	print(check)
	okt = Okt()
	ex_morpheme = []
	# 문장 형태소 분석
	for text in examples:
		mor_data = ""
		for word_tag in okt.pos(text, norm=True, stem=True):
			if word_tag[1] in ['Noun', 'Verb', 'VerbPrefix', 'Adjective', 'Determiner', 'Adverb', 'Exclamation', 'KoreanParticle']:
				mor_data += word_tag[0]
				mor_data += " "
		ex_morpheme.append(mor_data)
	# 문장 및 사용될 단어의 최대 길이/갯수 설정
	maxlen = 50
	max_words = 10000
	# 생성된 형태소 배열 토큰화 및 정수 인코딩
	tokenizer = Tokenizer(num_words=max_words)
	with open('wordIndex.json') as json_file:
		word_index = json.load(json_file)
		tokenizer.word_index = word_index
	tokenizer.fit_on_texts(word_index) # 빈도수 기준으로 단어 집합 생성
	sequences = tokenizer.texts_to_sequences(ex_morpheme) # 단어 빈도수가 높은 순으로 번호 부여
	# 문장 별 길이를 동일하게 맞추기 위한 패딩
	x_test = pad_sequences(sequences, maxlen=maxlen)
	# 모델을 불러와 데이터로부터 욕설 가능성 계산 후 반환
	model = load_model('model.h5', custom_objects = {"precision": precision, "recall" : recall, "f1score" : f1score})
	prob = model.predict(x_test)
	for i in range(0, len(x_test)):
		print(examples[i], ":", round(prob[i][0] * 100, 1), "%의 확률로 욕설입니다.")            
	prob_json = { "prob": []}
	for i in range(0, len(x_test)):
		prob_json["prob"].append(float(prob[i][0]))
	return prob_json

# Server address
if __name__ == "__main__":
	app.run(host="127.0.0.1", port="5000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


[False, False, False, False]

 : 19.7 %의 확률로 욕설입니다.
 
	 : 19.7 %의 확률로 욕설입니다.

		 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

		 : 19.7 %의 확률로 욕설입니다.

		 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

		 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

			
	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.
[if IE 7]>
	<link rel="stylesheet" type="text/css" href="//nstatic.dcinside.com/dc/w/css/ie7.css"/>
	<![endif] : 19.7 %의 확률로 욕설입니다.

		 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.
[if lt IE 9]>
	<script src="//nstatic.

세나 다리우스 오히려 베릴이 저거 보고 해보는거 아니냐 룰러가 단식세나 하냐 : 36.5 %의 확률로 욕설입니다.
03.16 23:15:35 : 19.7 %의 확률로 욕설입니다.
삭제 : 19.0 %의 확률로 욕설입니다.
ㅇㅇ : 35.3 %의 확률로 욕설입니다.
(223.38) : 19.7 %의 확률로 욕설입니다.
베릴때문에 좆된거노? : 93.7 %의 확률로 욕설입니다.
03.16 23:19:36 : 19.7 %의 확률로 욕설입니다.
삭제 : 19.0 %의 확률로 욕설입니다.
ㅇㅇ : 35.3 %의 확률로 욕설입니다.
(175.214) : 19.7 %의 확률로 욕설입니다.
코릴 '다리우스' 못막는다 : 17.5 %의 확률로 욕설입니다.
03.17 01:31:41 : 19.7 %의 확률로 욕설입니다.
삭제 : 19.0 %의 확률로 욕설입니다.
ㅇㅇ : 35.3 %의 확률로 욕설입니다.
(211.36) : 19.7 %의 확률로 욕설입니다.
근데 바텀이 캐리했네ㅋㅋㅋㅋ : 47.3 %의 확률로 욕설입니다.
03.16 23:16:03 : 19.7 %의 확률로 욕설입니다.
삭제 : 19.0 %의 확률로 욕설입니다.
ㅇㅇ : 35.3 %의 확률로 욕설입니다.
(59.14) : 19.7 %의 확률로 욕설입니다.
생각해보면 렐 상대로 ㄹㅇ 괜찮을수도 있겠네 들어오면 죽이니까 : 19.9 %의 확률로 욕설입니다.
03.16 23:16:11 : 19.7 %의 확률로 욕설입니다.
삭제 : 19.0 %의 확률로 욕설입니다.
ㅇㅇ : 35.3 %의 확률로 욕설입니다.
(211.214) : 19.7 %의 확률로 욕설입니다.
비디디는 솔랭에서도 아지르하네 ㅋㅋㅋㅋ : 73.1 %의 확률로 욕설입니다.
03.16 23:16:17 : 19.7 %의 확률로 욕설입니다.
삭제 : 19.0 %의 확률로 욕설입니다.
ㅇㅇ : 35.3 %의 확률로 욕설입니다.
(180.66) : 19.7 %의 확률로 욕설입니다.
응디디 솔랭 전적 보니깐 이렐이 모스트고 아지르 몇판 없던데? : 89.8 %의 확

										 : 19.7 %의 확률로 욕설입니다.

																				 : 19.7 %의 확률로 욕설입니다.

										 : 19.7 %의 확률로 욕설입니다.

										 : 19.7 %의 확률로 욕설입니다.

										 : 19.7 %의 확률로 욕설입니다.

										 : 19.7 %의 확률로 욕설입니다.

									 : 19.7 %의 확률로 욕설입니다.
2598233 : 19.7 %의 확률로 욕설입니다.

																																			 : 19.7 %의 확률로 욕설입니다.

																								
																																												 : 19.7 %의 확률로 욕설입니다.
																					 : 19.7 %의 확률로 욕설입니다.
왜 하필 젠지, 담원이  까일까? : 18.0 %의 확률로 욕설입니다.
[98] : 19.7 %의 확률로 욕설입니다.

											 : 19.7 %의 확률로 욕설입니다.
																					 : 19.7 %의 확률로 욕설입니다.
ㅇㅇ : 35.3 %의 확률로 욕설입니다.
(211.214) : 19.7 %의 확률로 욕설입니다.
03.16 : 19.7 %의 확률로 욕설입니다.
26682 : 19.7 %의 확률로 욕설입니다.
663 : 19.7 %의 확률로 욕설입니다.

										 : 19.7 %의 확률로 욕설입니다.

																				 : 19.7 %의 확률로 욕설입니다.

										 : 19.7 %의 확률로 욕설입니다.

										 : 19.7 %의 확률로 욕설입니다.

										 : 19.7 %의 확률로 욕설입니다.

										 : 19.7 %의 확률로 욕설입니다.

									 : 19.7 %의 확률로 욕설입니다.
2598149 : 19.7 %의 확률로 

...

 19.7 %의 확률로 욕설입니다.

	   : 19.7 %의 확률로 욕설입니다.

		   : 19.7 %의 확률로 욕설입니다.
/ : 19.7 %의 확률로 욕설입니다.

		 : 19.7 %의 확률로 욕설입니다.
1 : 19.7 %의 확률로 욕설입니다.
6 : 19.7 %의 확률로 욕설입니다.
이전 : 18.9 %의 확률로 욕설입니다.
다음 : 19.2 %의 확률로 욕설입니다.

		 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

			 : 19.7 %의 확률로 욕설입니다.

			 : 19.7 %의 확률로 욕설입니다.

			 : 19.7 %의 확률로 욕설입니다.

		 : 19.7 %의 확률로 욕설입니다.

			   : 19.7 %의 확률로 욕설입니다.

			 : 19.7 %의 확률로 욕설입니다.

			   : 19.7 %의 확률로 욕설입니다.

			 : 19.7 %의 확률로 욕설입니다.
애틀란타 총격 추가정보 : 17.7 %의 확률로 욕설입니다.

	<a class="inner logClass"  href="${link}" depth1="rightframe" depth2="issuezoom">
		<span class="img_box">
		  <img src="${original_link}" alt="" width="272">
		</span>
		<div class="txt_box">
		  <strong>${title}</strong>
		</div>
		<span class="issue_inner"></span>
	</a>
	 : 19.7 %의 확률로 욕설입니다.

   : 19.7 %의 확률로 욕설입니다.

 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	
	 : 19.7 %의 확률로 욕설입니다.

	 : 19.7 %의 확률로 욕설입니다.

	
   : 19.7 %의 확률로 욕설입니다.

	   : 19.7 %의 확률로 욕설입니다.

	   : 19.7 %의 확률

[False, False, False, False]


127.0.0.1 - - [17/Mar/2021 15:47:20] "POST /use_model HTTP/1.1" 200 -


가쓰오부시 - 나무위키 : 27.4 %의 확률로 욕설입니다.
.resize-observer[data-v-b329ee4c]{position:absolute;top:0;left:0;z-index:-1;width:100%;height:100%;border:none;background-color:transparent;pointer-events:none;display:block;overflow:hidden;opacity:0}.resize-observer[data-v-b329ee4c] object{display:block;position:absolute;top:0;left:0;height:100%;width:100%;overflow:hidden;pointer-events:none;z-index:-1} : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
최근 변경 : 18.3 %의 확률로 욕설입니다.
최근 토론 : 16.8 %의 확률로 욕설입니다.
특수 기능 : 18.3 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
게시판 : 19.7 %의 확률로 욕설입니다.
작성이 필요한 문서 : 22.8 %의 확률로 욕설입니다.
고립된 문서 : 17.9 %의 확률로 욕설입니다.
분류가 되지 않은 문서 : 14.7 %의 확률로 욕설입니다.
편집된 지 오래된 문서 : 14.4 %의 확률로 욕설입니다.
내용이 짧은 문서 : 17.6 %의 확률로 욕설입니다.
내용이 긴 문서 : 16.7 %의 확률로 욕설입니다.
차단 내역 : 19.5 %의 확률로 욕설입니다.
RandomPage : 19.7 %의 확률로 욕설입니다.
파일 올리기 : 18.1 %의 확률로 욕설입니다.
라이선스 : 19.7 %의 확률로 욕설입니다.
아카 : 38.3 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
112.168.241.107 : 19.7 %

[False, False, False, False]
